In [4]:
import findspark
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [3]:
sc = spark.sparkContext

### Almacenamiento en caché

El almacenamiento en caché permite que Spark conserve los datos en todos los cálculos y operaciones.\
Es una de las técnicas más importantes de spark para acelerar los cálculos, especialmente cuando se trata de cálculos interactivos.\
Funciona almacenando el RDD tanto como sea posible en la memoria. Si los datos que se solicitan para almacenar en caché son más grandes que la memoria disponible, el rendimiento disminuirá porque se utilizará disco en lugar de memoria.

* Podemos marcar un RDD como almacenado en caché usando **persist()** o **cache()**

    * **cache()** es un sinónimo de persist(MEMORY_ONLY)
    * **persist()** puede usar memoria, disco o ambos

#### Valores posibles para el nivel de almacenamiento

* **MEMORY_ONLY** almacena el RDD como un objeto Java deserialización en la JVM. Si el RDD no cabe memoria, algunas particiones no se almacenarán en caché y se volverán a calcular sobre la marcha cada vez que se necesiten. Este es el nivel por defecto en spark.

* **MEMORY_AND_DISK** almacena los RDD como objetos Java deserializados en la JVM. Si el RDD no cabe en memoria, almacena las particiones que no quepan en el disco y las lee desde allí cuando sea necesario.

* **DISK_ONLY** almacena las particiones del RD solo en disco.

* **MEMORY_ONLY_2, MEORY_AND_DISK_2, etc** igual que los niveles anteriores, pero replica cada partición en los nodos del cluster.

#### ¿Qué nivel de almacenamiento elegir?
El nivel de almacenamiento a elegir depende de la situación:

* Si los RDD caben en la memoria, use **MEMORY_ONLY**, ya que es la opción más rápida para el rendimiento de ejecución.

* **DISK_ONLY** no debe usarse a menos que sus cálculos sean costosos.

*  Utilice almacenamiento replicado para una mejor tolerancia fallos si puede ahorrar la memoria adicional necesaria. Esto evitará que se vuelvan a calcular las particiones perdidas para obtener la mejor disponibilidad.

>Podemos usar la función **unpersist()** para liberar el contenido en cahé

In [5]:
rdd = sc.parallelize([item for item in range(10)])

In [6]:
rdd.persist(StorageLevel.MEMORY_ONLY)

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:287

>\
>Si quisiéramos cambiar el nivel de persistencia al mismo RDD, debemos tener en cuenta que primero es necesario hacerle **unpersist()** a este mismo RDD. De lo contrario, si tratamos de cambiarle el nivel de persistencia sin realizar esta acción, nos va a devolver un error.
>
><br>

In [ ]:
rdd.unpersist()

rdd.persist(StorageLevel.DISK_ONLY)

In [ ]:
rdd.unpersist()
# Cuando le aplicamos cache(), estamos haciendo un MEMORY_ONLY
rdd.cache()

### Partition y shuffling (mezcla de datos)

* Los RDD operan con datos no como una sola masa de datos, sino que administran y operan los datos en particiones repartidas por todo el cluster. Por lo tanto, el concepto de partición de datos es fundamental para el correcto funcionamiento de los chop de Apache spark y puede tener un gran efecto en el rendimiento y en la forma en que se utilizan los recursos.

* Los RDD constan de particiones de datos y todas las operaciones se realizan en las particiones de datos en el RDD. Algunas operaciones, como las **transformaciones**, son funciones ejecutadas por un ejecutor en la partición específica de datos en la que se opera. Sin embargo, no todas las operaciones pueden realizarse simplemente realizando operaciones aisladas en las particiones de datos por parte de los respectivos ejecutores Las operaciones como las **agregaciones**, requieren que los datos se muevan a través del cluster en una fase conocida como mezcla o **shuffle**.

#### Importancia del número de particiones

* Si la cantidad de particiones es demasiado pequeña, usaremos sólo unas pocas CPU o núcleos en una gran cantidad de datos, por lo que tendremos un rendimiento más lento y dejaremos el cluster subutilizado.

* Si la cantidad de particiones es demasiado grande, utilizará más recursos de los que realmente necesita y en un entorno de múltiples procesos, podría estar provocando la falta de recursos para otros procesos que usted u otros miembros de su equipo ejecutan.

#### Particionadores

El particionamiento de los RDD se realiza mediante patinadores. Los patinadores asignan un índice de partición a los elementos del RDD. Todos los elementos de la misma partición tendrán el mismo índice de partición.\
Spark dispone de dos patinadores el HashPartitioner y el RangePartitioner.
Además de esto, también puede implementar un patrocinador personalizado.

* **HashPartitioner:** es el patrocinador predeterminado en spark y funciona calculando un valor hash para cada clave de los elementos. Todos los elementos con el mismo código hash terminan en una misma partición y esto se hace utilizando la fórmula siguiente:

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;partitionIndex = hash(key) % numPartitions

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;El partitionIndex es iguel al hash del elemento dividido entre el número de particiones

* **RangePartitioner:** funciona dividiendo el RDD en rangos aproximadamente iguales. Dado que el rango debe conocer las claves de inicio y final de cualquier partición, el RDD debe ordenarse primero antes de que se pueda usar un RangePartitioner. RangePartitioner primero necesita límites razonables para las particiones basadas en el RDD, luego crea una función desde la clave K hasta el partitionIndex al que pertenece el elemento y finalmente necesitamos reparticionar el RDD, basado en el RangePartitioner y para redistribuir los elementos del RDD correctamente según los rangos que determinamos.

#### Veamos en la práctica cómo podemos implementar un **HashPartitioner** manualmente.

In [7]:
rdd2 = sc.parallelize(['x', 'y', 'z'])

In [8]:
hola = 'Hola'

In [9]:
# Obtenemos el valor hash de la variable hola
hash(hola)

514654146388779058

In [10]:
num_particiones = 6

In [11]:
# indice = hash(item) % num_particiones

print(hash('x') % num_particiones)

print(hash('y') % num_particiones)

print(hash('z') % num_particiones)

# Aquí podemos ver a qué partición irá cada valor

4
3
3


#### Shuffling (mezcla de datos)

Cualquiera que sea el participio utilizado, muchas operaciones provocarán un reparticionamiento de datos en las particiones de un RDD. Se pueden crear nuevas particiones o se pueden contraer o fusionar varias particiones.

Todo el movimiento de datos necesario para el reparto se denomina **shuffling** y este es un concepto importante que hay que comprender al escribir un job de Spark.

El shuffling puede causar un gran impacto en el rendimiento, ya que los cálculos ya no están en la memoria del mismo ejecutor, sino que los ejecutores están intercambiando datos a través de la red.

Cuando un RDD está experimentando una transformación, se intenta que las operaciones que se realicen en el mismo nodo que los datos. Sin embargo, a menudo utilizamos operaciones de unión, reducción, agrupación o agregación, entre otras, que provocan el shuffling intencionado o no intencionado.

Este shuffle, a su vez, determina dónde ha finalizado una etapa particular del procesamiento y donde ha comenzado una nueva.

>\
>Cuanto más suffling tengamos, más etapas o stages ocurren en la ejecución del Job que afecta el rendimiento. Por lo tanto, estos temas debemos tenerlos en cuenta a la hora de construir de nuestros Jobs de Spark.
>
><br>


### Broadcast variables

Las variables broadcast son variables compartidas entre todos los ejecutores. Éstas se crean una vez en el controlador y luego se leen sólo en los ejecutores.

Se pueden transmitir conjuntos de datos completos en un cluster de Spark para que los ejecutores tengan acceso a los datos transmitidos. Todas las tareas que se ejecutan dentro de un ejecutor tienen acceso a las variables broadcast.

Lo que sucede al crear una variable broadcast es que la información de esta variable estará disponible en todos los **worker nodes** y será posible acceder a los datos de esta misma variable broadcast.

In [12]:
uno = 1

In [13]:
# Creamos una variable broadcast llamando a la función broadcast del sparl context
br_uno = sc.broadcast(uno)

In [14]:
# Vamos a sumar al primer rdd que creamos el valor de la variable broadcast, llamándola con .value
rdd3 = rdd.map(lambda x: x + br_uno.value)

rdd3.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

>\
>Las variables broadcast ocupan memoria en todos los ejecutores y según el tamaño de los datos contenidos en la variable broadcast, estos puede causar problemas de recursos en algún momento.
>
><br>

Existe una forma de eliminar las variables de la memoria de todos los ejecutores.
Para ello, lo que vamos a hacer es llamar a la función **unpersist()** en una variable que previamente lavtengamos como broadcast y esto lo que va a hacer es eliminar los datos de la variable broadcast de la memoria de caché de todos los ejecutores para así liberar recursos.

Si la variable se volviera a utilizar los datos se retransmitirían a los ejecutores para que se vuelvan a utilizar.

Tenemos que tener en cuenta que después de llamar a unpersist(), si accedemos a la variable broadcast nuevamente funciona como de costumbre, es decir, por detrás de escena los executors están extrayendo los datos de la variable nuevamente.

In [15]:
# Al aplicar unpersist() retiramos la variable de los executors
br_uno.unpersist()

In [16]:
# Si volvemos a llamar a la broadcast variable a la que le acabamos de aplicar el unpersist(), se ejecuta sin ningún tipo de problema
rdd4 = rdd.map(lambda x: x + br_uno.value)

rdd4.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

>\
>Para eliminar completamente una variable broadcast, hay que eliminarla de todos los executors y del driver con la función **destroy()**
>
>Esto puede resultar muy útil para administrar los recursos de manera óptima en todo el cluster.
>
><br>

In [17]:
# destroy()  destruye todos los datos y metadatos relacionados con la variable broadcast especificada.
br_uno.destroy()

In [ ]:
# Si ahora intentamos volver a acceder a la variable broadcast, nos arrojará un error
rdd5  = rdd.map(lambda x: x + br_uno.value)

rdd5.take(5)

### Accumulator variables

Los acumuladores son variables compartidas entre los ejecutores, que normalmente se utilizan para agregar contadores a su programa en Spark.

* **sparkContext.accumulator()** se usa para definir variables de acumulador

* **add()** se usa para agregar/actualizar un valor en el acumulador

* La propiedad **.value** de la variable del acumulador se utiliza para recuperar el valor del acumulador.

In [22]:
# Primero creamos un accumulator dándole como parámetro el valor con el cual deseamos iniciar el acumulador
acumulador = sc.accumulator(0)

In [23]:
rdd6 = sc.parallelize([2,4,6,8,10])

In [24]:
# Ahora vamos a calcular suma total de los elementos que componen el RDD usando el accumulator
# Lo que vamos a hacer es ir sumando al accumulator cada uno de los valores del RDD
rdd6.foreach(lambda x: acumulador.add(x))

In [25]:
# Comprobamos el valor del accumulator
print(acumulador.value)

30


In [26]:
# También podemos usar accumulator para saber la cantidad de elementos que contiene el RDD
rdd1 = sc.parallelize('Mi nombre es Miguel y me siento genial'.split(' '))

acumulador1 = sc.accumulator(0)

rdd1.foreach(lambda x: acumulador1.add(1))

print(acumulador1.value)
# Y 8 será la cantidad de palabras contenidas en la frase

8
